In [2]:
import torch

In [5]:
import torch

!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu117.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 28.1 MB/s eta 0:00:0000:010:01
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu117.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 88.1 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 44.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 67.9 MB/s eta 0:00:0000:0100:01
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu117.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 85.4 MB/s eta 0:00:00:00:01
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-bi097ur1
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-bi097ur1
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit 5778c65686b0c6af40e745a3ec449e7b27628ead
  Installing build depen

In [6]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

In [14]:
from gcn_explain_mcts import explain_reg_mcts
from model.prototree import GCNProtoSoftTree
from utils.draw import draw_last_layer, draw_similarity
from utils.featurizer import from_smiles
from utils.train_test import test, train
import torch.nn.functional as F

In [15]:
import copy
import numpy as np
from rdkit import Chem
from rdkit.Chem.rdmolops import GetAdjacencyMatrix
import torch
from sklearn.cluster import KMeans
from sklearn.metrics import mean_absolute_error
from torch import nn
# from torch.utils.tensorboard import SummaryWriter
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

from args import get_args
from features.GCN import GCN, GCNResidual

from gcn_explain_mcts import explain_reg_mcts
from model.prototree import GCNProtoSoftTree
from utils.draw import draw_last_layer, draw_similarity
from utils.featurizer import from_smiles
from utils.train_test import test, train
import torch.nn.functional as F
from tdc.benchmark_group import admet_group



from Alchemy_dataset import TencentAlchemyDataset
from torch_geometric.nn import GCNConv, Set2Set

import pandas as pd
from validate import get_results, get_valid_results, get_valid_targets, get_valid_dataset
import sys
# import wandb
from tqdm import tqdm

In [ ]:
class x(object):

    def __init__(self):
        pass
args = x()
args.features_size = 1024
args.classes = 16
args.depth = 5
args.metric = 'mae'
args.dataset_name = "LD50"

args.lr = 0.005
args.cuda = 0
args.cl = 0.1
args.ld = 0.01
args.pt = 0.01

args.epoch = 400
args.batch_size = 64
args.prototype_size = 128
args.latent_distance = "parents"
args.project_start = 250
args.project_mod = 25
args.warmup = 150
args.join = 200
args.split_seed = 1
args.dataset_name = "herg"
args.split_seed = 1
args.split_method = 'scaffold'
args.task_type = "clst"

: 

In [3]:
train_dataset = TencentAlchemyDataset(root='data/data-bin', mode='dev').shuffle()
valid_dataset = TencentAlchemyDataset(root='data/data-bin', mode='valid')

train_dataset.data.y = train_dataset.data.y[:,0].view(-1,1)

valid_targets = get_valid_targets(root='data/data-bin/raw')
valid_dataset = get_valid_dataset(valid_targets, valid_dataset)

train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
proj_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=args.batch_size)
test_loader = DataLoader(valid_dataset, batch_size=args.batch_size)

['data/data-bin/processed/TencentAlchemy_dev.pt']
['data/data-bin/processed/TencentAlchemy_valid.pt']


In [4]:
args.task_type = "clst"

gnn_model = GCN(15, [256, 512, 512, 1024])

features = gnn_model.features

protoTree: GCNProtoSoftTree = GCNProtoSoftTree(features, args)
protoTree.train()

protoTree.join()
lr = args.lr * 10
lr_add, lr_features, lr_protos, lr_leaves = lr * 2 ** (-args.depth), lr * 2 ** (-args.depth), lr, lr

parameters = [
    {'params': protoTree.features.parameters(), 'lr': lr_features},
    {'params': protoTree.add_on.parameters(), 'lr': lr_add},
    #    {'params': protoTree.prototype_vectors, 'lr': lr_protos},
    {'params': protoTree.leaves, 'lr': lr_leaves}
]

for i in range(protoTree.depth):
    depth = protoTree.depth - i - 1
    parameters.append({'params': protoTree.prototypes[i], 'lr': lr_protos * (2 ** (-depth))})

optimizer = torch.optim.Adam(parameters)
pt, cl,  ld = args.pt, args.cl, args.ld

train_y = np.array([x.y.item() for x in train_loader.dataset])
#  mean = train_y.mean()
#  std = train_y.std()
#  train_y = (train_y-mean)/std
mean = 0
std = 1
kmeans = KMeans(n_clusters=args.classes).fit(train_y.reshape(-1, 1))
labels = kmeans.predict(train_y.reshape(-1, 1))
weights = []
for i in range(args.classes):
    weights.append(1/(len(labels[labels == i])/len(labels)))

weights = torch.Tensor(weights)
weights = weights/weights.max()
#weights = weights/weights.norm()
#criterion = nn.NLLLoss(weight=weights.to(device))
criterion = nn.NLLLoss()

In [5]:
total_parts_train, total_stats_train = train(protoTree, criterion, train_loader, optimizer, None,task_type=args.task_type, pt=pt, cl=cl, ld=ld, kmeans=kmeans, mean=mean, std=std)

Starting Training!


0it [00:00, ?it/s][W NNPACK.cpp:51] Could not initialize NNPACK! Reason: Unsupported hardware.
10it [00:26,  2.63s/it]


In [6]:
criterion = nn.MSELoss()
args.task_type = "reg"
args.classes = 1
mean = 0
std = 1

old_protoTree = protoTree
leaves = F.softmax(old_protoTree.leaves, dim=1).cpu().detach().numpy()
leaves = (leaves * kmeans.cluster_centers_.reshape(-1)).sum(-1).reshape(-1, 1)

protoTree: GCNProtoSoftTree = GCNProtoSoftTree(features, args)
for i in range(protoTree.depth):
    protoTree.prototypes[i] = nn.Parameter(old_protoTree.prototypes[i].data, requires_grad=True)

protoTree.leaves = nn.Parameter(torch.Tensor(leaves), requires_grad=True)
# protoTree.to(device)

lr = args.lr
lr_add, lr_features, lr_protos, lr_leaves = lr * 2 ** (-args.depth), lr * 2 ** (-args.depth), lr, lr

parameters = [
    {'params': protoTree.features.parameters(), 'lr': lr_features},
    {'params': protoTree.add_on.parameters(), 'lr': lr_add},
    #    {'params': protoTree.prototype_vectors, 'lr': lr_protos},
    {'params': protoTree.leaves, 'lr': lr_leaves}
]

for i in range(protoTree.depth):
    depth = protoTree.depth - i - 1
    parameters.append({'params': protoTree.prototypes[i], 'lr': lr_protos * (2 ** (-depth))})

optimizer = torch.optim.Adam(parameters)
protoTree.warmup()

best = 999999
best_proj = 99999

In [7]:
epoch = 150

In [8]:
if epoch >= args.project_start and epoch % args.project_mod == 0:
    protoTree.eval()
    protoTree.project_fast_prototypes(proj_loader)
    protoTree.train()
    protoTree.last_only()

    for proj_idx in range(15):
        _, _ = train(protoTree, criterion, train_loader, optimizer, device, task_type=args.task_type, pt=pt, cl=cl, ld=ld,  mean=mean, std=std)

    total_parts_eval, total_stats_eval = test(protoTree, criterion, valid_loader, optimizer, device, task_type=args.task_type, pt=pt, cl=cl, ld=ld, mean=mean, std=std)
    total_parts_test, total_stats_test = test(protoTree, criterion, test_loader, optimizer, device, task_type=args.task_type, pt=pt, cl=cl, ld=ld, mean=mean, std=std)

    if epoch >= args.project_start and total_stats_eval[args.metric] < best_proj:
        best_proj = total_stats_eval[args.metric]
        print(f"BEST! {best_proj} - {total_stats_test[args.metric]}")

    torch.save(protoTree.state_dict(), f'./saves/{args.dataset_name}_{model_name}_proj.pt')


    if epoch >= args.join:
        protoTree.join()
    else:
        protoTree.warmup()


    total_parts_train, total_stats_train = train(protoTree, criterion, train_loader, optimizer, device,  task_type=args.task_type, pt=pt, cl=cl,  ld=ld, mean=mean, std=std)